### 목표 설정
1. 앞서 만든 NPZ 파일을 불러와서 사용해보자
2. CNN(합성곱 신경망)을 구현해보자
3. 신경망 성능 개선 및 전이 학습을 진행해보자

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd

/content


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/딥러닝

/content/drive/MyDrive/Colab Notebooks/딥러닝


In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [6]:
# npz 파일 로딩
data = np.load('data/cats_dogs.npz')
data

NpzFile 'data/cats_dogs.npz' with keys: X_train, X_test, y_train, y_test

In [7]:
# 데이터 분할
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2000, 224, 224, 3), (1000, 224, 224, 3), (2000,), (1000,))

### CNN 모델링
1. 모델 설계 및 구축
2. 학습 및 평가 방법 설정
3. 학습 및 시각화
4. 모델 성능 평가

In [10]:
# 재료 import
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [11]:
# 1. 신경망 구현
# 뼈대 설정
model = Sequential()

# 특성 추출부 구현
model.add(Conv2D(filters=32,
                 kernel_size=(3, 3),
                 padding='same',     # 패딩 사용하겠다
                 activation='relu',
                 input_shape=(224, 224, 3)))

model.add(MaxPooling2D(pool_size=(2,2)))  # input이 짝수라서 pool_size도 짝수
# ===============================================
model.add(Conv2D(filters=64,
                 kernel_size=(3, 3),
                 padding='same',
                 activation='relu'
                 ))

model.add(MaxPooling2D(pool_size=(2,2)))
# ================================================
model.add(Conv2D(filters=128,
                 kernel_size=(3, 3),
                 padding='same',
                 activation='relu'
                 ))

model.add(MaxPooling2D(pool_size=(2,2)))
# ===============================================
# 특성 추출부 구현 완료

# 전결합층 구현
model.add(Flatten())

model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.35))

model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.35))

model.add(Dense(units=1, activation='sigmoid'))
# 전결합층 구현 완료

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 28, 28, 128)       0

In [13]:
# 2. 학습 및 평가 방법 설정
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
# 학습 조기중단 구현
f_early = EarlyStopping(monitor='val_accuracy',
                        patience=5)

In [16]:
# 3. 모델 학습
h = model.fit(X_train, y_train,
              epochs=100,
              batch_size=64,
              validation_split=0.2,
              callbacks=[f_early])

Epoch 1/100
25/25 [==============================] - 3s 112ms/step - loss: 0.2631 - accuracy: 0.8913 - val_loss: 1.3739 - val_accuracy: 0.3825
Epoch 2/100
25/25 [==============================] - 3s 103ms/step - loss: 0.2183 - accuracy: 0.9156 - val_loss: 1.2769 - val_accuracy: 0.5125
Epoch 3/100
25/25 [==============================] - 3s 111ms/step - loss: 0.1677 - accuracy: 0.9431 - val_loss: 1.1714 - val_accuracy: 0.5375
Epoch 4/100
25/25 [==============================] - 3s 113ms/step - loss: 0.1362 - accuracy: 0.9588 - val_loss: 1.2423 - val_accuracy: 0.5300
Epoch 5/100
25/25 [==============================] - 3s 103ms/step - loss: 0.1045 - accuracy: 0.9756 - val_loss: 1.1609 - val_accuracy: 0.5700
Epoch 6/100
25/25 [==============================] - 3s 110ms/step - loss: 0.0813 - accuracy: 0.9800 - val_loss: 1.5425 - val_accuracy: 0.5400
Epoch 7/100
25/25 [==============================] - 3s 110ms/step - loss: 0.0646 - accuracy: 0.9862 - val_loss: 1.5168 - val_accuracy: 0.5625

- 학습 중간에 표시되는 정확도와 검증 정확도를 확인한 결과 과대적합 발생!
- 과대적합 제어 방법이 Dropout과 학습 조기 중단을 반영해도 과대적합이 해소되지 않았다
- 데이터가 모자라서 학습이 부족할 수 있다. 데이터를 증강해보자!

### 데이터 증식
- 부족한 데이터를 원본 데이터를 이용해서 증강시켜주는 방법
- 일반적으로 딥러닝 모델은 데이터가 많을수록 성능이 좋아진다

In [17]:
# 이미지 증식 진행 해보기
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    rescale = 1./255,  # 데이터 스케일링 -> 현재는 연산을 통해 minmax 스케일링 구현
    rotation_range=15, # 회전 각도 -> 0~360도 사이의 각도 지정
    width_shift_range=0.1,   # 이미지를 좌우로 이동 (10% 내외)
    height_shift_range=0.1,  # 이미지를 상하로 이동 (10% 내외)
    zoom_range=[0.8, 2.0],   # 0.8배 또는 2배로 확대 및 축소
    shear_range=0.5,         # 0.5라디안 내외 시계 방향으로 변형(약 28.6도)
    horizontal_flip=True,    # 좌우 반전
    vertical_flip=True,      # 상하 반전
    fill_mode='nearest'      # 보간법 : 이미지 픽셀의 빈 공간을 채워주는 방법
)

test_gen = ImageDataGenerator(rescale=1./255)
# test 데이터는 이미지 증식이 필요 없음

In [22]:
# train / test 경로 잡아주기
train_dir = 'data/cats_and_dogs_filtered/train'
test_dir = 'data/cats_and_dogs_filtered/test'

In [23]:
# 이미지 증식 시작
# train_flow_directory() : 폴더의 경로 설정 및 옵션을 부여
train_generator = train_gen.flow_from_directory(train_dir, # 폴더 경로 지정
                                                target_size=(224,224), # 변환할 이미지 사이즈
                                                batch_size=64,    # 한번에 변환시킬 이미지 개수
                                                class_mode='binary') # 라벨 번호 0번부터 시작 / 폴더 알파벳 순서대로 읽어오기

test_generator = test_gen.flow_from_directory(test_dir,
                                              target_size=(224,224),
                                              batch_size=10,
                                              class_mode='binary'
                                                )

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [24]:
# 라벨링 결과 확인
print(train_generator.class_indices)
print(test_generator.class_indices)

{'cats': 0, 'dogs': 1}
{'cats': 0, 'dogs': 1}


### CNN 신경망 구현

In [25]:
# 뼈대 설정
model2 = Sequential()
# 특성 추출부 구현
# conv2D_1 layer
# filter 갯수 32 / 패딩 사용 / 필터 사이즈 : (3,3) / 활성화 함수 relu / input_shape 지정
model2.add(Conv2D(filters = 32,
                  kernel_size = (3,3),
                  activation = 'relu',
                  input_shape = (224, 224, 3),
                  padding = 'same'))

# MaxPooling layer
# pool_size = (2,2)
model2.add(MaxPooling2D(pool_size = (2,2)))

# conv2D_2 layer
# filter 갯수 64 / 패딩 사용 / 필터 사이즈 : (3,3) / 활성화 함수 relu
model2.add(Conv2D(filters = 64,
                  kernel_size = (3,3),
                  activation = 'relu',
                  padding = 'same'))

# MaxPooling layer
# pool_size = (2,2)
model2.add(MaxPooling2D(pool_size = (2,2)))
# conv2D_3 layer
# filter 갯수 128 / 패딩 사용 / 필터 사이즈 : (3,3) / 활성화 함수 relu
model2.add(Conv2D(filters = 128,
                  kernel_size = (3,3),
                  activation = 'relu',
                  padding = 'same'))

# MaxPooling layer
# pool_size = (2,2)
model2.add(MaxPooling2D(pool_size = (2,2)))

# 전결합층 구현
# 데이터를 1차원으로 펴주는 층
model2.add(Flatten())
# 뉴런의 갯수 64 / 활성화 함수 : 렐루
model2.add(Dense(units = 64, activation = 'relu'))
# 드롭아웃 추가(비워두기)
model2.add(Dropout(0.35))
# 출력층 설정 (이진분류에 맞는 출력층 설정)
model2.add(Dense(units = 1, activation = 'sigmoid'))

model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 112, 112, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 56, 56, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 28, 28, 128)      

In [26]:
model2.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
# 학습 조기중단 구현
# 이미지 증식을 진행했을 때는 학습이 들쭉날쭉하게 진행이 된다.
# 학습횟수와 기회를 늘려서 학습을 오래 시켜줄 필요가 있다.
f_early = EarlyStopping(monitor='val_accuracy',
                        patience=20)

epochs = 100

h2 = model2.fit_generator(generator = train_generator,
                          epochs = epochs,
                          validation_data = test_generator,
                          callbacks = [f_early]
                          )

<ipython-input-27-8a2f3e07f993>:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  h2 = model2.fit_generator(generator = train_generator,


Epoch 1/100
32/32 [==============================] - 553s 17s/step - loss: 0.7005 - accuracy: 0.4935 - val_loss: 0.6892 - val_accuracy: 0.5790
Epoch 2/100
32/32 [==============================] - 41s 1s/step - loss: 0.6868 - accuracy: 0.5375 - val_loss: 0.6765 - val_accuracy: 0.6120
Epoch 3/100
32/32 [==============================] - 41s 1s/step - loss: 0.6718 - accuracy: 0.5840 - val_loss: 0.6615 - val_accuracy: 0.5990
Epoch 4/100
32/32 [==============================] - 42s 1s/step - loss: 0.6643 - accuracy: 0.5975 - val_loss: 0.6400 - val_accuracy: 0.6400
Epoch 5/100
32/32 [==============================] - 42s 1s/step - loss: 0.6502 - accuracy: 0.5970 - val_loss: 0.6299 - val_accuracy: 0.6460
Epoch 6/100
32/32 [==============================] - 41s 1s/step - loss: 0.6375 - accuracy: 0.6320 - val_loss: 0.6182 - val_accuracy: 0.6490
Epoch 7/100
32/32 [==============================] - 40s 1s/step - loss: 0.6353 - accuracy: 0.6290 - val_loss: 0.6183 - val_accuracy: 0.6410
Epoch 8/100

### 전이학습
- 기존에 학습이 잘 된 모델을 이용하는 방법
- 전이학습에는 특성 추출 방식과 미세 조정 방식이 있음


In [28]:
# vgg16 모델 import
from tensorflow.keras.applications import VGG16

In [29]:
vgg16 = VGG16(
    include_top=False, # 불러온 모델의 MLP 분류기를 쓸건가?
    weights='imagenet', # imagenet에서 사용했ㄷ너 1000개 클래스에 대응하는 가중치를 사용하겠다(사전학습 가중치 사용)
    input_shape = (224, 224, 3) # 이미지 사이즈
    )

58889256/58889256 [==============================] - 0s 0us/step


In [30]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [31]:
# 특성 추출부에 MLP를 결합해주자
# 우리가 실습할 부분은 미세 조정 방식
# 동결 시킬 층을 확인하기 위해 불러온 VGG16 모델의 층의 이름을 확인해보자
for layer in vgg16.layers:
  print(layer.name)

In [32]:
# 신경망 설계
# 뼈대 설정
transfer_model = Sequential()

# 미세 조정 방식 적용
# 분류기와 맞닿은 block5_conv3층만 학습이 가능하도록 설정
for layer in vgg16.layers:
  if layer.name == 'block5_conv3':
    layer.trainable = True
  else :
    layer.trainable = False

# 모델 구현
transfer_model.add(vgg16)

transfer_model.add(Flatten())

transfer_model.add(Dense(units=128, activation='relu'))

# 출력층
transfer_model.add(Dense(units=1, activation='sigmoid'))

In [33]:
# 모델 컴파일
transfer_model.compile(optimizer='adam',
                       loss = 'binary_crossentropy',
                       metrics=['accuracy']
                       )

In [34]:
# 모델 학습
transfer_model.fit(X_train, y_train,
                   epochs=10,
                   validation_split=0.3)

Epoch 1/10
44/44 [==============================] - 21s 309ms/step - loss: 3.4121 - accuracy: 0.8907 - val_loss: 0.4863 - val_accuracy: 0.8417
Epoch 2/10
44/44 [==============================] - 8s 181ms/step - loss: 0.0821 - accuracy: 0.9786 - val_loss: 0.4875 - val_accuracy: 0.9083
Epoch 3/10
44/44 [==============================] - 8s 182ms/step - loss: 0.0739 - accuracy: 0.9850 - val_loss: 0.5022 - val_accuracy: 0.9200
Epoch 4/10
44/44 [==============================] - 8s 175ms/step - loss: 0.0326 - accuracy: 0.9900 - val_loss: 0.2870 - val_accuracy: 0.9517
Epoch 5/10
44/44 [==============================] - 8s 183ms/step - loss: 0.0151 - accuracy: 0.9964 - val_loss: 0.5586 - val_accuracy: 0.9117
Epoch 6/10
44/44 [==============================] - 8s 184ms/step - loss: 0.0115 - accuracy: 0.9964 - val_loss: 0.6128 - val_accuracy: 0.9417
Epoch 7/10
44/44 [==============================] - 8s 177ms/step - loss: 0.0057 - accuracy: 0.9993 - val_loss: 0.8629 - val_accuracy: 0.9350
Epoch

In [35]:
transfer_model.evaluate(X_test, y_test)

32/32 [==============================] - 6s 184ms/step - loss: 0.3668 - accuracy: 0.9650


[0.36682578921318054, 0.9649999737739563]

### 주의점
- 한번 학습 가능하도록 설정된 층은 가중치가 변경
- 만약 동결층을 바꿔서 다시 학습 시키고 싶다면 처음부터 모델을 새롭게 import 해와야 한다.